In [1]:
# Relevant module imports and installs
%pip install pulp pandas brotli fuzzywuzzy
import pandas as pd
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import json


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the data retrieval functions
from challenge_data import get_bootstrap_static_data, get_gameweek_data, update_with_gameweek_cost

# Get the actual GW11 data
bootstrap_data = get_bootstrap_static_data()
gw11_data = update_with_gameweek_cost(get_gameweek_data(11, bootstrap_data), '2024-25', 11)

Error loading gameweek 11: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/gws/gw11.csv


In [3]:
# Create optimization model
model = plp.LpProblem("GW11_Hindsight_Optimization", plp.LpMaximize)

# Get number of players
player_count = len(gw11_data)

# Create binary variables for lineup and captain selection
lineup = [plp.LpVariable(f"lineup_{i}", 0, 1, plp.LpBinary) for i in range(player_count)]
captain = [plp.LpVariable(f"captain_{i}", 0, 1, plp.LpBinary) for i in range(player_count)]

# Objective function: Maximize total points (double for captain)
model += plp.lpSum([lineup[i] * gw11_data.loc[i, 'Points'] + captain[i] * gw11_data.loc[i, 'Points'] for i in range(player_count)])

# Constraints
# Exactly 5 players
model += plp.lpSum(lineup) == 5

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if gw11_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# At least 1 Defender
model += plp.lpSum([lineup[i] for i in range(player_count) if gw11_data.loc[i, 'Position'] == 'Defender']) >= 1

# At least 1 Midfielder
model += plp.lpSum([lineup[i] for i in range(player_count) if gw11_data.loc[i, 'Position'] == 'Midfielder']) >= 1

# At least 1 Forward
model += plp.lpSum([lineup[i] for i in range(player_count) if gw11_data.loc[i, 'Position'] == 'Forward']) >= 1

# Solve the problem
model.solve(plp.PULP_CBC_CMD(msg=False))

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Points: {points}")
                total_points += points
    print(f"\nTotal Points: {total_points}")

# Print the results
print("Status:", plp.LpStatus[model.status])

print("\nOptimal Lineup:")
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = gw11_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Points': player['Points'],
            'Captain': captain[i].value() == 1
        })

print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Pickford - Everton - Points: 11

Defender:
  Tarkowski - Everton - Points: 11

Midfielder:
  M.Salah - Liverpool - Points: 17
  B.Fernandes (C) - Man Utd - Points: 40

Forward:
  Wissa - Brentford - Points: 19

Total Points: 98
